In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
import matplotlib.pyplot as plt
import glob
from sklearn import svm
import cv2
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import to_categorical
import os
path_train='../input/signdata/data/asl_alphabet_train'
train = list(glob.glob(path_train+'/*/*.jpg'))
labels = list(map(lambda x:os.path.split(os.path.split(x)[0])[1], train))
numeric_labels=[]
for l in labels:
    if l =='del':
        numeric_labels.append(26)
    elif l=='nothing':
        numeric_labels.append(27)
    elif l=='space':
        numeric_labels.append(28)
    else:
        numeric_labels.append(ord(l.upper())-ord('A'))
train = pd.Series(train, name='FilePath').astype(str)
labels = pd.Series(numeric_labels, name='Label')
data = pd.concat([train, labels], axis=1)
data = data.sample(frac=1).reset_index(drop=True)
Xtrain=[]
Ytrain=[]
Xval=[]
Yval=[]
Xtest=[]
Ytest=[]
print(len(train))

for t in range(int(len(train)*.7)) :
    img=cv2.imread(data['FilePath'][t])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(100,100))
    #img= cv2.GaussianBlur(img, (3, 3), 0)
    #img=cv2.Canny(image=img, threshold1=3, threshold2=100)
    Xtrain.append(img)
    Ytrain.append(data['Label'][t])
for t in range(int(len(train)*.7),int(len(train)*.9)):
    img=cv2.imread(data['FilePath'][t])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(100,100))
    #img= cv2.GaussianBlur(img, (3, 3), 0)
    #img=cv2.Canny(image=img, threshold1=3, threshold2=100)
    Xtest.append(img)
    Ytest.append(data['Label'][t])
for t in range(int(len(train)*.9),len(train)):
    img=cv2.imread(data['FilePath'][t])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(100,100))
    #img= cv2.GaussianBlur(img, (3, 3), 0)
    #img=cv2.Canny(image=img, threshold1=3, threshold2=100)
    Xval.append(img)
    Yval.append(data['Label'][t])
print(set(Ytrain))
Xtrain=np.array(Xtrain)
Xval=np.array(Xval)
Xtest=np.array(Xtest)
#Xtrain=np.expand_dims(Xtrain, axis=3)
#Xval=np.expand_dims(Xval, axis=3)
#Xtest=np.expand_dims(Xtest, axis=3)
print(Xval.shape)
print(len(to_categorical(Yval)))
print(Xtrain.shape)
print(len(to_categorical(Ytrain)))
model = models.Sequential()
model.add(layers.Conv2D(100, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.Conv2D(50, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((3,3 )))
model.add(layers.Conv2D(100, (3, 3), activation='relu'))
model.add(layers.Conv2D(120, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(75, (3, 3), activation='relu'))
model.add(layers.Conv2D(50, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(29,activation='softmax'))
model.summary()

model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
history=model.fit(
  Xtrain,
  to_categorical(Ytrain),
  epochs=10,
  validation_data=(Xval, to_categorical(Yval)),
)
model.save("model_2.h5", include_optimizer=True)
Ypred=model.predict(Xtest)
ypred=[]
for i in Ypred:
    for c in range(len(i)):
        if i[c]==max(i):
            ypred.append(c)
print(len(ypred))
print(len(Ypred))
print(accuracy_score(Ytest,ypred))
print(precision_score(Ytest,ypred,average=None))
print(recall_score(Ytest,ypred,average=None))